In [4]:
# task 1:

# https://randomuser.me/documentation

# Topic per nationality
# Partition by gender

import confluent_kafka
import requests
from confluent_kafka import Producer
from confluent_kafka.admin import AdminClient, NewTopic
import time
import datetime

def acked(err, msg):
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))

def fetch()-> list:
    url = "https://randomuser.me/api"

    params = dict()
    params["results"] = "2"
    params["nat"] = "US"

    for i in range(10):
        r = requests.get(url, params=params)
        if r.status_code != 200:
            if i==9:
                print("API failed after 10 tries")
                return None
            time.sleep(1)
        else:
            break

    return r.json()["results"]

def produce_events(events:list):
    conf = {'bootstrap.servers': "localhost:19092,localhost:29092"}
    producer = Producer(conf)

    for i in events:
        topic = i["nat"] # we want to have each nationality in its own topic
        key = i["gender"] # we want to partition by gender
        producer.produce(topic=topic, key=key, value=str(i), callback=acked)

    producer.poll(1)
    producer.flush() 





In [5]:
import time 

while True:
    users = fetch()  
    produce_events(users)
    time.sleep(4)